# dAiv 순환환신경망 특강: 시퀀스 데이터의 압축과 생성 [한영 번역기편편]

이번 특강에서는 한국어 문장을 영어로 번역하는 RNN 시퀀스-투-시퀀스(sequence-to-sequence, seq2seq) 모델을 학습하는 방법을 알아보겠습니다.

필요 라이브러리: ``torchtext``, ``spacy`` 를 사용하여 데이터셋을 전처리(preprocess)합니다.
> 이 예제에서 torchtext는 huggingface의 tokenizers로 대체하여 구현되었습니다.

## Import

### For Local User

In [ ]:
from platform import system

%pip install uv
!uv init
!uv sync

if system() == "Windows":
    %uv add torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
else:
    %uv add torch torchvision torchaudio

%uv add matplotlib tqdm numpy pandas scipy jupyter ipywidgets
%uv add git+https://github.com/dAiv-CNU/torchdaiv.git

### For Colab User

In [ ]:
%pip add matplotlib tqdm numpy pandas scipy jupyter ipywidgets
%pip install git+https://github.com/dAiv-CNU/torchdaiv.git

### Download Datasets

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm

#### Library Imports

In [1]:
import torch
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import spacy
from torchdaiv import datasets
from torchdaiv.lectures.kor_eng_translator import nn
from torchdaiv.lectures.kor_eng_translator.util import vocabulary, transforms

from transformers import GPT2Tokenizer, GPT2LMHeadModel

from tqdm.auto import tqdm
from rich.traceback import install
#install(show_locals=True)  # 오류 났을 경우 로컬 변수 보고 싶으면 활성화

%matplotlib inline

c:\Users\BREW\Desktop\marimo_test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
## Text Preprocess (with Spacy)

``torchtext`` 에는 언어 변환 모델을 만들 때 쉽게 사용할 수 있는 데이터셋을 만들기 적합한 다양한 도구가 있습니다.
이 예제에서는 가공되지 않은 텍스트 문장(raw text sentence)을 토큰화(tokenize)하고, 어휘집(vocabulary)을 만들고,
토큰을 텐서로 숫자화(numericalize)하는 방법을 알아보겠습니다.

| (다만, torchtext는 2024년 4월 이후 더 이상 업데이트가 진행되지 않는다는 점에 유의해야 합니다.)

아래를 실행하여 Spacy 토크나이저가 쓸 한국어와 영어에 대한 데이터를 다운로드 받습니다.

In [2]:
# spacy tokenizer 적용
ko_tokenizer = vocabulary.load_tokenizer(spacy, "ko_core_news_sm")
en_tokenizer = vocabulary.load_tokenizer(spacy, "en_core_web_sm")

In [3]:
from spacy.lang.ko.examples import sentences

# 작동 확인
doc = spacy.load("ko_core_news_sm")(sentences[0])
print("Original:", doc.text)
print("Tokenized:", ko_tokenizer(sentences[0]), end="\n\n")

for token in doc:
    print(">", token.text, f"({token.lemma_}) |", token.pos_, token.dep_)

Original: 애플이 영국의 스타트업을 10억 달러에 인수하는 것을 알아보고 있다.
Tokenized: ['애플', '##이', '영국', '##의', '스타트업', '##을', '10', '##억', '달러', '##에', '인수', '##하', '##는', '것', '##을', '알아보고', '있', '##다', '.']

> 애플이 (애플+이) | NOUN dislocated
> 영국의 (영국+의) | PROPN nmod
> 스타트업을 (스타트업+을) | ADV advcl
> 10억 (10+억) | NUM compound
> 달러에 (달러+에) | NOUN advcl
> 인수하는 (인수+하+는) | VERB acl
> 것을 (것+을) | NOUN obj
> 알아보고 (알아보고) | AUX ROOT
> 있다 (있+다) | AUX aux
> . (.) | PUNCT punct


---
## Load Dataset
using spacy

In [4]:
# 데이터셋 로드 - 아무 처리도 하지 않았을 때
# 이번 수업에서는 트레인 데이터셋으로만 사용

train_dataset = datasets.AnkiKorEngDataset("./data", split_rate=(1.0, 0.0, 0.0))
# valid_dataset = datasets.AnkiKorEngDataset("./data", valid=True, split_rate=(0.5, 0.3, 0.2))
# test_dataset = datasets.AnkiKorEngDataset("./data", test=True, split_rate=(0.5, 0.3, 0.2))

Extraction completed.
Dataset loaded. 5822 samples loaded.


In [5]:
# 데이터셋 형태 확인
sample = list(zip(*train_dataset[0:5]))+list(zip(*train_dataset[500:505]))
for i, (kor, eng) in enumerate(sample):
    print(i, kor, eng)

0 가. Go.
1 안녕. Hi.
2 뛰어! Run!
3 뛰어. Run.
4 누구? Who?
5 저건 뭐야? What's that?
6 누가 그를 그렸습니까? Who drew it?
7 쉬엄쉬엄 일해. Work slowly.
8 너 늦었어. You're late.
9 넌 내 거야. You're mine.


#### Vocabulary 생성

In [6]:
ko_vocab = vocabulary.build_vocab(raw_dataset=train_dataset.raw_kor, tokenizer=ko_tokenizer)
en_vocab = vocabulary.build_vocab(raw_dataset=train_dataset.raw_eng, tokenizer=en_tokenizer)

#### Convert To Tensor

In [7]:
# 사전 데이터를 기반으로 데이터셋을 텐서로 변환
to_tensor = (
    transforms.to_tensor(ko_vocab, tokenizer=ko_tokenizer),
    transforms.to_tensor(en_vocab, tokenizer=en_tokenizer)
)

train_dataset.transform(transform=to_tensor)
# valid_dataset.transform(transform=to_tensor)
# test_dataset.transform(transform=to_tensor)

Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1
Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1


In [8]:
# 데이터셋 형태 확인
sample = list(zip(*train_dataset[0:5]))+list(zip(*train_dataset[500:505]))
for i, (kor, eng) in enumerate(sample):
    print(i, kor, eng)

0 tensor([53,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0]) tensor([25,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  

#### Data Loader

In [9]:
# 배치 크기 결정 후 데이터 로더 생성
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# valid_dataload = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset)//20)

---
# PRACTICE 2: (N-to-N(or M)): 한영 번역기 (RNN, GRU)
---

---
## Model Definition
> 인코더 역할을 수행하는 RNN/GRU 하나만 사용하였던 실습 1과는 달리, 레이어를 여러 층으로 쌓는 방식의 Encoder to Decoder 모델을 사용

> 인풋과 아웃풋의 길이가 정해져 있는 경우가 아니라면 어떻게 해야할 지에 대한 해결책!

![RNN 구조](https://jiho-ml.com/content/images/2020/04/figure3-2.png)

> `인코더`는 `한국어를 해석`하고, `디코더`는 `영어를 생성`하는 방식으로 역할을 나눠서 번역을 수행

> -> 인코더에서 정보를 압축하고, 디코더가 원하는 크기로 압축을 해제


> 참고사항:
>> 아래 예시 모델은 공부하기 쉬운 단순한 모델로 번역에 있어 매우 뛰어난 성능을 보이는 모델은 아닙니다.
>> 최신 기술 트렌드는 Transformers를 사용하는 것입니다.
>> 혹시 관심이 있다면 [Transformer 레이어](https://pytorch.org/docs/stable/nn.html#transformer-layers)에 대해 더 알아보시기 바랍니다.

In [10]:
# 사전(보캡) 등록
nn.set_vocabulary(ko_vocab, en_vocab)

5499 2525 2525


In [11]:
# 모델 정의
class Seq2Seq(nn.Module):
    def __init__(self, height=2, hidden=128):
        super(Seq2Seq, self).__init__()
        # 인코더와 디코더를 정의합니다.
        self.encoder = nn.Encoder(nn.GRU, height=height, hidden=hidden, dropout=0.3)  # nn.RNN으로도 바꿔보세요.
        self.decoder = nn.Decoder(nn.GRU, height=height, hidden=hidden, dropout=0.3)  # nn.RNN으로도 바꿔보세요.

    def forward(self, korean, english):
        context_vector = self.encoder(korean)
        output = self.decoder(english, context_vector)
        return output

In [12]:
# 하이퍼 파라미터 설정
epoch = 20
lr = 1e-4  # learning rate
height = 2
hidden = 128

In [13]:
# 모델 생성
model = Seq2Seq(height=height, hidden=hidden)
model.init_optimizer(lr=lr)
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5499, 256)
    (model): GRU(256, 128, num_layers=2, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (embedding): Embedding(2525, 256)
    (model): GRU(256, 128, num_layers=2, batch_first=True, dropout=0.3)
    (fc1): Linear(in_features=128, out_features=1262, bias=True)
    (fc2): Linear(in_features=1262, out_features=2525, bias=True)
  )
  (criterion): CrossEntropyLoss()
)

In [14]:
# 가능한 경우 쿠다 사용
if torch.cuda.is_available:
    model.cuda()
print("Use Device:", model.device)

Use Device: cuda


In [15]:
model.fit(train_dataloader, epoch)

epoch:1/20 loss: 0.20447806457241813
epoch:2/20 loss: 0.1993886289688257
epoch:3/20 loss: 0.1982472327711818
epoch:4/20 loss: 0.19748997802917773
epoch:5/20 loss: 0.19661885974826393
epoch:6/20 loss: 0.19581917648787026
epoch:7/20 loss: 0.19549074376022424
epoch:8/20 loss: 0.19530960868348132
epoch:9/20 loss: 0.1951195708997957
epoch:10/20 loss: 0.19497203106408592
epoch:11/20 loss: 0.19484122359490658
epoch:12/20 loss: 0.1947005373108518
epoch:13/20 loss: 0.1945701030256984
epoch:14/20 loss: 0.194445992400358
epoch:15/20 loss: 0.1943988683787021
epoch:16/20 loss: 0.19421729838455115
epoch:17/20 loss: 0.19413333050497286
epoch:18/20 loss: 0.19405292842414354
epoch:19/20 loss: 0.19397959620743008
epoch:20/20 loss: 0.19393013286721575


## Translation Test

In [16]:
model.translate('좋은 아침!', transform=transforms.to_tensor(ko_vocab, tokenizer=ko_tokenizer))

Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1
buy wear a wrong . a . <bos> a bank . 

---
# PRACTICE 3: 생성(N-to-M): 한영 번역기 (GPT)
---

---
## Model Definition
> Encoder와 Decoder가 이해의 편의와 인지 부담을 낮추기 위한 도구였고, 실제로는 Decoder만을 통한 정보 처리도 가능함을 확인 (편견의 극복)

> -> 인코더를 통한 정보 압축을 하지 않고 정보를 한번에 다 디코더에 넣는다면 어떨까? 그리고 출력을 한번에 하지 않고 반복한다면 어떨까?

![어텐션 스코어](https://slds-lmu.github.io/seminar_nlp_ss20/figures/02-02-attention-and-self-attention-for-nlp/bahdanau-fig3.png)

![GPT vs Llama](https://miro.medium.com/v2/resize:fit:1132/1*zdLBI0pShQlgHujAyG_g3g.png)

In [17]:
# GPT-2 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 트랜스토머 모델을 위해 데이터셋 변환 필요요
class AdapterDataset(Dataset):
    def __init__(self, texts, tokenizer, block_size=64):
        self.examples = [
            tokenizer(
                text, truncation=True, max_length=block_size, padding="max_length", return_tensors="pt"
            ).input_ids.squeeze() for text in texts if text.strip()
        ]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx], self.examples[idx]

In [ ]:
# Load gpt2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # reflect pad_token
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Initialize optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Train
for epoch in range(1):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}")

    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(input_ids=inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"[Epoch {epoch+1}] Loss: {avg_loss:.4f}")

## Translation Test

In [ ]:
model.eval()

# Generate text using the Huggingface model.
prompt = "The future of AI is"
inputs = tokenizer(prompt, return_tensors="pt", padding=True)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

output_ids = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_new_tokens=50,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Generated Text: {generated_text}")